In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from networkx.algorithms import bipartite
from google.colab import files

%matplotlib inline

In [ ]:
url = 'https://github.com/kmlee419/patent_analysis/raw/main/tesla_ipc_new.csv'
df = pd.read_csv(url)

In [ ]:
url1 = 'https://github.com/kmlee419/cpc_search/raw/main/cpc_total1.csv'
url2 = 'https://github.com/kmlee419/cpc_search/raw/main/cpc_total2.csv'

df1 = pd.read_csv(url1)
df2 = pd.read_csv(url2)

df_cpc = pd.concat([df1, df2])

In [ ]:
df['code_1'] = df['code'].str[:1]
df['code_2'] = df['code'].str[:3]

### 총 특허 수

In [ ]:
len(df['patent'].drop_duplicates())

### 총 코드 수

In [ ]:
len(df['code'].drop_duplicates())

### 대상년도

In [ ]:
print(min(df['date']), max(df['date']))

### 연도별 총 특허 수 

In [ ]:
df2 = df[['patent', 'year']].drop_duplicates()
group_year = df2.groupby('year')['patent'].count()
group_year = group_year.reset_index()

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data=group_year, x='year', y='patent')
plt.show()

### 가장 많이 사용된 코드 top N list (세세분류)

In [ ]:
df_cpc.head()

In [ ]:
year1 = 2016 # 기간설정
year2 = 2021

df_year = df[(df['year']>=year1)&(df['year']<=year2)]
code_count = df_year['code'].value_counts().reset_index()
df_cpc2 = df_cpc.rename(columns={'코드': 'index'})
code_count = pd.merge(code_count, df_cpc2, on='index', how='left')
code_count.head(10) # 순위 개수 바꾸기 가능

### 가장 많이 사용된 코드 top N list (중분류)

In [ ]:
year1 = 2016 # 기간설정
year2 = 2021

df_year = df[(df['year']>=year1)&(df['year']<=year2)]
code_count = df_year['code_2'].value_counts().reset_index()
df_cpc2 = df_cpc.rename(columns={'코드': 'index'})
code_count = pd.merge(code_count, df_cpc2, on='index', how='left')

code_count.head(10) # 순위 개수 바꾸기 가능

### 가장 많이 사용된 코드 top N list (대분류 > 중분류 > 세세분류)

In [ ]:
year1 = 2016 # 기간설정
year2 = 2016

df_year = df[(df['year']>=year1)&(df['year']<=year2)]
code_count = df_year['code_1'].value_counts().reset_index()
df_cpc2 = df_cpc.rename(columns={'코드': 'index'})
code_count = pd.merge(code_count, df_cpc2, on='index', how='left')

code_count.head(10) # 순위 개수 바꾸기 가능

In [ ]:
#대분류 선택
code = 'G'

df_code = df[df['code_1']==code]
code_count = df_code['code_2'].value_counts().reset_index()
df_cpc2 = df_cpc.rename(columns={'코드': 'index'})
code_count = pd.merge(code_count, df_cpc2, on='index', how='left')

code_count.head(10) # 순위 개수 바꾸기 가능

In [ ]:
#중분류 선택
code = 'G01'

df_code = df[df['code_2']==code]
code_count = df_code['code'].value_counts().reset_index()
df_cpc2 = df_cpc.rename(columns={'코드': 'index'})
code_count = pd.merge(code_count, df_cpc2, on='index', how='left')

code_count.head(10) # 순위 개수 바꾸기 가능

### 코드별 연도변화 (세세분류)

In [ ]:
code = 'E04D1/30' # 코드 바꾸기 가능

years = df[['year']].drop_duplicates().sort_values(by='year').reset_index(drop=True)

df_code = df[df['code']==code]
df_code2 = df_code[['patent', 'year']].drop_duplicates()
code_year = df_code2.groupby('year')['patent'].count()
code_year = code_year.reset_index()

code_year2 = pd.merge(years, code_year, on='year', how='left')
code_year2.fillna(0.0, inplace=True)

plt.figure(figsize=(10, 5))
sns.lineplot(data=code_year2, x='year', y='patent')
plt.show()

### 코드별 연도변화 (대분류)

In [ ]:
code = 'H' # 코드 바꾸기 가능

years = df[['year']].drop_duplicates().sort_values(by='year').reset_index(drop=True)

df_code = df[df['code_1']==code]
df_code2 = df_code[['patent', 'year']].drop_duplicates()
code_year = df_code2.groupby('year')['patent'].count()
code_year = code_year.reset_index()

code_year2 = pd.merge(years, code_year, on='year', how='left')
code_year2.fillna(0.0, inplace=True)

plt.figure(figsize=(10, 5))
sns.lineplot(data=code_year2, x='year', y='patent')
sns.lineplot(data=group_year, x='year', y='patent')
plt.legend(['code', 'total'])

plt.show()

In [ ]:
code_year2

### 코드별 연도변화 (중분류)

In [ ]:
code = 'A47' # 코드 바꾸기 가능

years = df[['year']].drop_duplicates().sort_values(by='year').reset_index(drop=True)

df_code = df[df['code_2']==code]
df_code2 = df_code[['patent', 'year']].drop_duplicates()
code_year = df_code2.groupby('year')['patent'].count()
code_year = code_year.reset_index()

code_year2 = pd.merge(years, code_year, on='year', how='left')
code_year2.fillna(0.0, inplace=True)

plt.figure(figsize=(10, 5))
sns.lineplot(data=code_year, x='year', y='patent')
sns.lineplot(data=group_year, x='year', y='patent')
plt.legend(['code', 'total'])

plt.show()

### 코드별 전년대비 변화량 (세세분류)

In [ ]:
years = list(df['year'].drop_duplicates().sort_values())

diff = pd.DataFrame(columns = ['index', 'diff', 'by_year'])
for i in range(0, len(years)-1):
    year1 = years[i]
    year2 = years[i+1]
    
    df_year1 = df[df['year']==year1]
    code_count1 = df_year1['code'].value_counts().reset_index()
    df_year2 = df[df['year']==year2]
    code_count2 = df_year2['code'].value_counts().reset_index()
    
    merge = pd.merge(code_count1, code_count2, on='index', how='outer')
    merge = merge.fillna(0)
    merge['diff'] = merge['code_y'] - merge['code_x']
    merge['by_year'] = str(year1) + "-" + str(year2)
    
    diff = pd.concat([diff, merge[['index', 'diff', 'by_year']]])

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'index'})
diff = pd.merge(diff, df_cpc2, on='index', how='left')

diff.sort_values(by='diff', ascending=False).head(10) # 순위 개수 바꾸기 가능

In [ ]:
# 역순
diff.sort_values(by='diff', ascending=True).head(10) # 순위 개수 바꾸기 가능

### 코드별 전년대비 변화량 (대분류)

In [ ]:
years = list(df['year'].drop_duplicates().sort_values())

diff = pd.DataFrame(columns = ['index', 'diff', 'by_year'])
for i in range(0, len(years)-1):
    year1 = years[i]
    year2 = years[i+1]
    
    df_year1 = df[df['year']==year1]
    code_count1 = df_year1['code_1'].value_counts().reset_index()
    df_year2 = df[df['year']==year2]
    code_count2 = df_year2['code_1'].value_counts().reset_index()
    
    merge = pd.merge(code_count1, code_count2, on='index', how='outer')
    merge = merge.fillna(0)
    merge['diff'] = merge['code_1_y'] - merge['code_1_x']
    merge['by_year'] = str(year1) + "-" + str(year2)
    
    diff = pd.concat([diff, merge[['index', 'diff', 'by_year']]])

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'index'})
diff = pd.merge(diff, df_cpc2, on='index', how='left')

diff.sort_values(by='diff', ascending=False).head(10) # 순위 개수 바꾸기 가능

In [ ]:
# 역순
diff.sort_values(by='diff', ascending=True).head(10) # 순위 개수 바꾸기 가능

### 코드별 전년대비 변화량 (중분류)

In [ ]:
years = list(df['year'].drop_duplicates().sort_values())

diff = pd.DataFrame(columns = ['index', 'diff', 'by_year'])
for i in range(0, len(years)-1):
    year1 = years[i]
    year2 = years[i+1]
    
    df_year1 = df[df['year']==year1]
    code_count1 = df_year1['code_2'].value_counts().reset_index()
    df_year2 = df[df['year']==year2]
    code_count2 = df_year2['code_2'].value_counts().reset_index()
    
    merge = pd.merge(code_count1, code_count2, on='index', how='outer')
    merge = merge.fillna(0)
    merge['diff'] = merge['code_2_y'] - merge['code_2_x']
    merge['by_year'] = str(year1) + "-" + str(year2)
    
    diff = pd.concat([diff, merge[['index', 'diff', 'by_year']]])

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'index'})
diff = pd.merge(diff, df_cpc2, on='index', how='left')

diff.sort_values(by='diff', ascending=False).head(10) # 순위 개수 바꾸기 가능

In [ ]:
# 역순
diff.sort_values(by='diff', ascending=True).head(10) # 순위 개수 바꾸기 가능

### 코드별 연평균 성장률(CAGR) (세세분류)

In [ ]:
df2 = df[['code', 'year', 'patent']].drop_duplicates()
df3 = df2.pivot_table(index='code', columns='year', values='patent', aggfunc=['count'])
df3.fillna(0.001, inplace=True)

In [ ]:
year1 = 2016
year2 = 2021

df3['cagr'] = (df3['count'][year2]/df3['count'][year1])**(1/(year2-year1))-1

In [ ]:
df3 = df3.reset_index()
df4 = df3[['code', 'cagr']].stack().reset_index(drop=True)
df_cpc2 = df_cpc.rename(columns={'코드': 'code'})
df5 = pd.merge(df4, df_cpc2, on='code', how='left')

In [ ]:
df5

In [ ]:
df5.sort_values(by='cagr', ascending=False).head(10)

### 코드별 연평균 성장률(CAGR) (대분류)

In [ ]:
df2 = df[['code_1', 'year', 'patent']].drop_duplicates()
df3 = df2.pivot_table(index='code_1', columns='year', values='patent', aggfunc=['count'])
df3.fillna(0.001, inplace=True)

In [ ]:
year1 = 2016
year2 = 2021

df3['cagr'] = (df3['count'][year2]/df3['count'][year1])**(1/(year2-year1))-1

In [ ]:
df3 = df3.reset_index()
df4 = df3[['code_1', 'cagr']].stack().reset_index(drop=True)
df_cpc2 = df_cpc.rename(columns={'코드': 'code_1'})
df5 = pd.merge(df4, df_cpc2, on='code_1', how='left')

In [ ]:
df5.sort_values(by='cagr', ascending=False).head(10)

### 코드별 연평균 성장률(CAGR) (중분류)

In [ ]:
df2 = df[['code_2', 'year', 'patent']].drop_duplicates()
df3 = df2.pivot_table(index='code_2', columns='year', values='patent', aggfunc=['count'])
df3.fillna(0.001, inplace=True)

In [ ]:
year1 = 2016
year2 = 2021

df3['cagr'] = (df3['count'][year2]/df3['count'][year1])**(1/(year2-year1))-1

In [ ]:
df3 = df3.reset_index()
df4 = df3[['code_2', 'cagr']].stack().reset_index(drop=True)
df_cpc2 = df_cpc.rename(columns={'코드': 'code_2'})
df5 = pd.merge(df4, df_cpc2, on='code_2', how='left')

In [ ]:
df5.sort_values(by='cagr', ascending=False).head(10)

### 전체 네트워크 분석 (세세분류)

In [ ]:
def network_generation(df):
    U_nodes = list(df['code'].drop_duplicates())
    V_nodes = list(df['patent'].drop_duplicates())
    edgelist = list(zip(df['code'], df['patent']))

    B_df = nx.Graph()
    B_df.add_nodes_from(U_nodes, bipartite=0)
    B_df.add_nodes_from(V_nodes, bipartite=1)

    B_df.add_edges_from(edgelist)
    G = bipartite.collaboration_weighted_projected_graph(B_df, U_nodes)

    return G

In [ ]:
G = network_generation(df)

In [ ]:
degree = nx.degree_centrality(G)
betweenness = nx.betweenness_centrality(G)
closeness = nx.closeness_centrality(G)
eigenvector = nx.eigenvector_centrality(G)

nx.set_node_attributes(G, degree, 'degree')
nx.set_node_attributes(G, betweenness, 'betweenness')
nx.set_node_attributes(G, closeness, 'closeness')
nx.set_node_attributes(G, eigenvector, 'eigenvector')

In [ ]:
df_g = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
df_g['dbratio'] = df_g['betweenness'] / df_g['degree']

In [ ]:
df_g = df_g.reset_index()
df_g.rename(columns={'index':'code'}, inplace=True)

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'code'})
df_g2 = pd.merge(df_g, df_cpc2, on='code', how='left')

In [ ]:
# degree 대비 betweenness가 높은 코드 순서로 출력
df_g2.sort_values(by='dbratio', ascending=False).head(10)

### 가장 빈도가 높은(낮은) 링크 

In [ ]:
year1 = 2016 # 기간설정
year2 = 2021

df_year = df[(df['year']>=year1)&(df['year']<=year2)]

G = network_generation(df_year)
df_g = nx.to_pandas_edgelist(G)

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
df_g2 = pd.merge(df_g, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
df_g3 = pd.merge(df_g2, df_cpc2, on='target', how='left')

In [ ]:
df_g3.sort_values(by='weight', ascending=False).head(10) # 순위 개수 바꾸기 가능

In [ ]:
df_g3.sort_values(by='weight', ascending=True).head(10) # 순위 개수 바꾸기 가능

### 가장 빈도가 높은(낮은) 링크 (대분류가 다른 링크만)

In [ ]:
year1 = 2016 # 기간설정
year2 = 2021

df_year = df[(df['year']>=year1)&(df['year']<=year2)]

G = network_generation(df_year)
df_g = nx.to_pandas_edgelist(G)

In [ ]:
df_g['source_1'] = df_g['source'].str[:1]
df_g['target_1'] = df_g['target'].str[:1]

In [ ]:
df_g = df_g[df_g['source_1']!=df_g['target_1']]

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
df_g2 = pd.merge(df_g, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
df_g3 = pd.merge(df_g2, df_cpc2, on='target', how='left')

In [ ]:
df_g3.sort_values(by='weight', ascending=False).head(10) # 순위 개수 바꾸기 가능

In [ ]:
df_g3.sort_values(by='weight', ascending=True).head(10) # 순위 개수 바꾸기 가능

### 링크별 전년대비 변화량

In [ ]:
years = list(df['year'].drop_duplicates().sort_values())

diff = pd.DataFrame(columns = ['source', 'target', 'diff', 'by_year'])
for i in range(0, len(years)-1):
    year1 = years[i]
    year2 = years[i+1]
    
    df_year1 = df[df['year']==year1]    
    df_year2 = df[df['year']==year2]

    G1 = network_generation(df_year1)
    G2 = network_generation(df_year2)
    
    df_g1 = nx.to_pandas_edgelist(G1)
    df_g2 = nx.to_pandas_edgelist(G2)
    
    df_merge = pd.merge(df_g1, df_g2, on=['source', 'target'], how='outer')
    df_merge = df_merge.fillna(0)
    df_merge['diff'] = df_merge['weight_y'] - df_merge['weight_x']
    df_merge['by_year'] = str(year1) + "-" + str(year2)
    
    diff = pd.concat([diff, df_merge[['source', 'target', 'diff', 'by_year']]])

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
diff2 = pd.merge(diff, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
diff3 = pd.merge(diff2, df_cpc2, on='target', how='left')

In [ ]:
diff3.sort_values(by='diff', ascending=False).head(20)

### 링크별 연평균 성장률(CAGR)

In [ ]:
year1 = 2016
year2 = 2021

df_year1 = df[df['year']==year1]    
df_year2 = df[df['year']==year2]

G1 = network_generation(df_year1)
G2 = network_generation(df_year2)
    
df_g1 = nx.to_pandas_edgelist(G1)
df_g2 = nx.to_pandas_edgelist(G2)

df_merge = pd.merge(df_g1, df_g2, on=['source', 'target'], how='outer')
df_merge = df_merge.fillna(0.001)
df_merge['cagr'] = (df_merge['weight_y']/df_merge['weight_x'])**(1/(year2-year1))-1

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
df_merge2 = pd.merge(df_merge, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
df_merge3 = pd.merge(df_merge2, df_cpc2, on='target', how='left')

In [ ]:
df_merge3.sort_values(by='cagr', ascending=False).head(10) # 순위 개수 바꾸기 가능

### 전체 네트워크 분석 (대분류)

In [ ]:
def network_generation2(df):
    U_nodes = list(df['code_1'].drop_duplicates())
    V_nodes = list(df['patent'].drop_duplicates())
    edgelist = list(zip(df['code_1'], df['patent']))

    B_df = nx.Graph()
    B_df.add_nodes_from(U_nodes, bipartite=0)
    B_df.add_nodes_from(V_nodes, bipartite=1)

    B_df.add_edges_from(edgelist)
    G = bipartite.collaboration_weighted_projected_graph(B_df, U_nodes)

    return G

In [ ]:
G = network_generation2(df)

In [ ]:
degree = nx.degree_centrality(G)
betweenness = nx.betweenness_centrality(G)
closeness = nx.closeness_centrality(G)
eigenvector = nx.eigenvector_centrality(G)

nx.set_node_attributes(G, degree, 'degree')
nx.set_node_attributes(G, betweenness, 'betweenness')
nx.set_node_attributes(G, closeness, 'closeness')
nx.set_node_attributes(G, eigenvector, 'eigenvector')

In [ ]:
df_g = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
df_g['dbratio'] = df_g['betweenness'] / df_g['degree']

In [ ]:
df_g = df_g.reset_index()
df_g.rename(columns={'index':'code_1'}, inplace=True)

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'code_1'})
df_g2 = pd.merge(df_g, df_cpc2, on='code_1', how='left')

In [ ]:
# degree 대비 betweenness가 높은 코드 순서로 출력
df_g2.sort_values(by='dbratio', ascending=False).head(10)

### 가장 빈도가 높은(낮은) 링크 

In [ ]:
year1 = 2016 # 기간설정
year2 = 2021

df_year = df[(df['year']>=year1)&(df['year']<=year2)]

G = network_generation2(df_year)
df_g = nx.to_pandas_edgelist(G)

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
df_g2 = pd.merge(df_g, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
df_g3 = pd.merge(df_g2, df_cpc2, on='target', how='left')

In [ ]:
df_g3.sort_values(by='weight', ascending=False).head(10) # 순위 개수 바꾸기 가능

In [ ]:
df_g3.sort_values(by='weight', ascending=True).head(10) # 순위 개수 바꾸기 가능

### 링크별 전년대비 변화량

In [ ]:
years = list(df['year'].drop_duplicates().sort_values())

diff = pd.DataFrame(columns = ['source', 'target', 'diff', 'by_year'])
for i in range(0, len(years)-1):
    year1 = years[i]
    year2 = years[i+1]
    
    df_year1 = df[df['year']==year1]    
    df_year2 = df[df['year']==year2]

    G1 = network_generation2(df_year1)
    G2 = network_generation2(df_year2)
    
    df_g1 = nx.to_pandas_edgelist(G1)
    df_g2 = nx.to_pandas_edgelist(G2)
    
    df_merge = pd.merge(df_g1, df_g2, on=['source', 'target'], how='outer')
    df_merge = df_merge.fillna(0)
    df_merge['diff'] = df_merge['weight_y'] - df_merge['weight_x']
    df_merge['by_year'] = str(year1) + "-" + str(year2)
    
    diff = pd.concat([diff, df_merge[['source', 'target', 'diff', 'by_year']]])

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
diff2 = pd.merge(diff, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
diff3 = pd.merge(diff2, df_cpc2, on='target', how='left')

In [ ]:
diff3.sort_values(by='diff', ascending=False).head(20)

### 링크별 연평균 성장률(CAGR)

In [ ]:
year1 = 2016
year2 = 2021

df_year1 = df[df['year']==year1]    
df_year2 = df[df['year']==year2]

G1 = network_generation2(df_year1)
G2 = network_generation2(df_year2)
    
df_g1 = nx.to_pandas_edgelist(G1)
df_g2 = nx.to_pandas_edgelist(G2)

df_merge = pd.merge(df_g1, df_g2, on=['source', 'target'], how='outer')
df_merge = df_merge.fillna(0.001)
df_merge['cagr'] = (df_merge['weight_y']/df_merge['weight_x'])**(1/(year2-year1))-1

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
df_merge2 = pd.merge(df_merge, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
df_merge3 = pd.merge(df_merge2, df_cpc2, on='target', how='left')

In [ ]:
df_merge3.sort_values(by='cagr', ascending=False).head(10) # 순위 개수 바꾸기 가능

### 전체 네트워크 분석 (중분류)

In [ ]:
def network_generation3(df):
    U_nodes = list(df['code_2'].drop_duplicates())
    V_nodes = list(df['patent'].drop_duplicates())
    edgelist = list(zip(df['code_2'], df['patent']))

    B_df = nx.Graph()
    B_df.add_nodes_from(U_nodes, bipartite=0)
    B_df.add_nodes_from(V_nodes, bipartite=1)

    B_df.add_edges_from(edgelist)
    G = bipartite.collaboration_weighted_projected_graph(B_df, U_nodes)

    return G

In [ ]:
G = network_generation3(df)

In [ ]:
degree = nx.degree_centrality(G)
betweenness = nx.betweenness_centrality(G)
closeness = nx.closeness_centrality(G)
eigenvector = nx.eigenvector_centrality(G)

nx.set_node_attributes(G, degree, 'degree')
nx.set_node_attributes(G, betweenness, 'betweenness')
nx.set_node_attributes(G, closeness, 'closeness')
nx.set_node_attributes(G, eigenvector, 'eigenvector')

In [ ]:
df_g = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')
df_g['dbratio'] = df_g['betweenness'] / df_g['degree']

In [ ]:
df_g = df_g.reset_index()
df_g.rename(columns={'index':'code_2'}, inplace=True)

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'code_2'})
df_g2 = pd.merge(df_g, df_cpc2, on='code_2', how='left')

In [ ]:
# degree 대비 betweenness가 높은 코드 순서로 출력
df_g2.sort_values(by='dbratio', ascending=False).head(10)

### 가장 빈도가 높은(낮은) 링크 

In [ ]:
year1 = 2016 # 기간설정
year2 = 2021

df_year = df[(df['year']>=year1)&(df['year']<=year2)]

G = network_generation3(df_year)
df_g = nx.to_pandas_edgelist(G)

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
df_g2 = pd.merge(df_g, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
df_g3 = pd.merge(df_g2, df_cpc2, on='target', how='left')

In [ ]:
df_g3.sort_values(by='weight', ascending=False).head(10) # 순위 개수 바꾸기 가능

In [ ]:
df_g3.sort_values(by='weight', ascending=True).head(10) # 순위 개수 바꾸기 가능

### 가장 빈도가 높은(낮은) 링크 (대분류가 다른 링크만)

In [ ]:
year1 = 2016 # 기간설정
year2 = 2021

df_year = df[(df['year']>=year1)&(df['year']<=year2)]

G = network_generation3(df_year)
df_g = nx.to_pandas_edgelist(G)

In [ ]:
df_g['source_1'] = df_g['source'].str[:1]
df_g['target_1'] = df_g['target'].str[:1]

In [ ]:
df_g = df_g[df_g['source_1']!=df_g['target_1']]

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
df_g2 = pd.merge(df_g, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
df_g3 = pd.merge(df_g2, df_cpc2, on='target', how='left')

In [ ]:
df_g3.sort_values(by='weight', ascending=False).head(10) # 순위 개수 바꾸기 가능

In [ ]:
df_g3.sort_values(by='weight', ascending=True).head(10) # 순위 개수 바꾸기 가능

### 링크별 전년대비 변화량

In [ ]:
years = list(df['year'].drop_duplicates().sort_values())

diff = pd.DataFrame(columns = ['source', 'target', 'diff', 'by_year'])
for i in range(0, len(years)-1):
    year1 = years[i]
    year2 = years[i+1]
    
    df_year1 = df[df['year']==year1]    
    df_year2 = df[df['year']==year2]

    G1 = network_generation3(df_year1)
    G2 = network_generation3(df_year2)
    
    df_g1 = nx.to_pandas_edgelist(G1)
    df_g2 = nx.to_pandas_edgelist(G2)
    
    df_merge = pd.merge(df_g1, df_g2, on=['source', 'target'], how='outer')
    df_merge = df_merge.fillna(0)
    df_merge['diff'] = df_merge['weight_y'] - df_merge['weight_x']
    df_merge['by_year'] = str(year1) + "-" + str(year2)
    
    diff = pd.concat([diff, df_merge[['source', 'target', 'diff', 'by_year']]])

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
diff2 = pd.merge(diff, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
diff3 = pd.merge(diff2, df_cpc2, on='target', how='left')

In [ ]:
diff3.sort_values(by='diff', ascending=False).head(20)

### 링크별 연평균 성장률(CAGR)

In [ ]:
year1 = 2016
year2 = 2021

df_year1 = df[df['year']==year1]    
df_year2 = df[df['year']==year2]

G1 = network_generation3(df_year1)
G2 = network_generation3(df_year2)
    
df_g1 = nx.to_pandas_edgelist(G1)
df_g2 = nx.to_pandas_edgelist(G2)

df_merge = pd.merge(df_g1, df_g2, on=['source', 'target'], how='outer')
df_merge = df_merge.fillna(0.001)
df_merge['cagr'] = (df_merge['weight_y']/df_merge['weight_x'])**(1/(year2-year1))-1

In [ ]:
df_cpc2 = df_cpc.rename(columns={'코드': 'source'})
df_merge2 = pd.merge(df_merge, df_cpc2, on='source', how='left')
df_cpc2 = df_cpc.rename(columns={'코드': 'target'})
df_merge3 = pd.merge(df_merge2, df_cpc2, on='target', how='left')

In [ ]:
df_merge3.sort_values(by='cagr', ascending=False).head(10) # 순위 개수 바꾸기 가능

### 해당 코드를 포함한 특허 추출

In [ ]:
code = 'G01C21/3697' # 코드 바꾸기 가능

df_code = df[df['code']==code].reset_index(drop=True)
df_code2 = df_code[['patent', 'date']].drop_duplicates().reset_index(drop=True)
df_code2['url'] = 'https://patents.google.com/patent/' + df_code['patent'].str.replace('-', '')

In [ ]:
df_code2

### 복수의 코드를 포함한 특허 추출

In [ ]:
code1 = 'H01M10/613' # 코드 바꾸기 가능
code2 = 'Y02E60/10' # 코드 바꾸기 가능

df_code1 = df[df['code']==code1].reset_index(drop=True)
df_code2 = df[df['code']==code2].reset_index(drop=True)
df_code3 = pd.merge(df_code1, df_code2, on='patent', how='outer')
df_code4 = df_code3[(~df_code3['code_x'].isnull())&(~df_code3['code_y'].isnull())]
df_code5 = df_code4[['patent', 'date_x']].reset_index(drop=True)
df_code5['url'] = 'https://patents.google.com/patent/' + df_code5['patent'].str.replace('-', '')

In [ ]:
df_code5